# CSRD DATASET GENERATOR #
_By: Michiel Tange_,
_Last updated: 5/11/2024_


The purpose of this script is to facilitate finding which companies in the areas of the Sparke & Keane offices can be contacted about CSRD.

It does this by first finding the location coordinates of all the company headquarters using Google Maps' Geocoding API. The company data is extracted from Factset. It then calculates the distances between the companies and the different Sparke & Keane offices. It adds the original Factset data, as well as the calculated data to an Excel output file. Afterwards, it sets up a criteria sheet in the Excel file where the criteria for consideration are defined. Then, each Sparke & Keane office gets added as a separate sheet to the output Excel file with the appropriate Excel formulas filled in for filtering the data according to the specified criteria. This ensures a dynamic file at the end that end-users can play around with.

### _TO DO_ ###
- convert to an OOP solution with the following objects:
    - Sparke & Keane office 
        - attributes: office parameters 
        - methods: distance calculations, Excel sheet adding
    - output file 
        - attributes: file parameters 
        - methods: open file, insert sheet, save file
    - criteria
        - attributes: classification (CSRD or S&K), default value, column pointer in Factset data, desired column name for output file
- add custom function(s) for facilitating the translations to Excel references (e.g., cel 'A1')
- add security features (e.g., locked cells) to the Excel file

## Set-up ##

### Imports ###

In [2]:
import string
import openpyxl
import googlemaps
import pandas as pd
from os import listdir
from haversine import haversine

### API key ###
Reading in the Google Cloud developer API key.

In [4]:
with open("APIkey.txt", 'r') as file:
    API_key = file.read()

### Client ###

In [275]:
gmaps = googlemaps.Client(key=API_key)

### Data files ###

In [ ]:
data_files = listdir("Factset data")

### Global parameters ###
Define custom functions to be used throughout the script.

In [175]:
def get_coords(location):
    """
    get the longitude and latitude coordinates for a location

    args:
        location (str): location for which to get coordinates
    
    returns: 
        coordinates (dict): dictionary containing longitude and latitude denoted by the keys 'lng' and 'lat'
    """ 
    return gmaps.geocode(location)[0]['geometry']['location']

Set the addresses of the different office of Sparke & Keane (can be added to / adjusted).

In [176]:
SK_Maastricht_address = 'Bassin 108, 6211 AK Maastricht, Netherlands'
SK_Eindhoven_address = 'Achtseweg Zuid 221, 5651 GW Eindhoven, Netherlands'
SK_Nijmegen_address = 'Oranjesingel 51, 6511 NP Nijmegen, Netherlands'

SK_Maastricht_loc = get_coords(SK_Maastricht_address)
SK_Eindhoven_loc = get_coords(SK_Eindhoven_address)
SK_Nijmegen_loc = get_coords(SK_Nijmegen_address)

SK_offices = {'SK Maastricht':  SK_Maastricht_loc,
              'SK Eindhoven':   SK_Eindhoven_loc,
              'SK Nijmegen':    SK_Nijmegen_loc} # dictionary with the Sparke & Keane offices locations (coordinates)

Set the parameters of the output (Excel) file.

In [280]:
output_file = "CSRD dataset.xlsx"
full_data_sheet = 'Ruwe data'
criteria_sheet = 'Criteria'

Set the parameters for the criteria. Criteria are a dictionary with the criteria name as key, and default value as value.

In [338]:
criteria = {# CSRD criteria
            'Minimaal aantal werknemers (#)':       250,
            'Minimale omzet (MM$)':                 50,
            'Minimale activa (MM$)':                25,
            # Sparke & Keane criteria
            'Maximale afstand tot kantoor (km)':    50} # always put CSRD criteria first, and Sparke & Keane criteria last

# the corresponding column names for each csrd criteria (from Factset)
csrd_criteria_cols = {  'Minimaal aantal werknemers (#)':   'Number of Employees',
                        'Minimale omzet (MM$)':             'Revenue (MM, USD)',
                        'Minimale activa (MM$)':            'Total Assets (MM) (USD) (USD)'}

## Pre-processing ##
Cleaning up some weird stuff Factset adds to the Excel file. <span style="color:green">**Run this only once.**</span>

In [265]:
for data_file in data_files:
    filename = data_file
    wb = openpyxl.load_workbook(filename)
    sheet = wb['Companies_Results']
    sheet.delete_rows(1, 3)
    wb.save(filename)

### Loading data ###

In [271]:
data = pd.DataFrame()
for data_file in data_files:
    temp_data = pd.read_excel(data_file)
    data = pd.concat([data, temp_data], ignore_index=True)

### Finding all unique cities ###

In [272]:
city_countries = data.loc[data['City'] != '-',['City', 'Country']].drop_duplicates() # save unique city-country pairs (some cities occur in multiple countries) - drop all '-' for missing cities
city_countries['City_Country'] = city_countries['City'] + ', ' + city_countries['Country'] # combine both pandas Series into one
cities = city_countries['City_Country'].to_numpy() # save as a numpy array for fast iterating

### Initializing empty dictionary ###
The dictionary is going to hold the cities as the keys, and their coordinates as the values. The coordinates are also a dictionary with keys: 'lng' & 'lat' for longitude and latitude respectively, and the actual coordinates as the values.

In [273]:
city_coords = {}

## Distances ##

### Getting the location coordinates ###
Getting the coordinates (longitude and latitude) for each of the unique cities using Google's Geocoding API.

In [276]:
for city in cities:
    try:
        coords = get_coords(city)
    except IndexError as e:
        print(f'{e} for city: {city}') # error handling for cities (city-country pairs) Google couldn't find
    city_coords[city] = coords


### Calculate the distances between cities and offices ###
Calculate the distances between a city location (coordinates) and the Sparke & Keane offices (coordinates). The distances are calculated using the haversine formula for calculating distances between points on a sphere:
$$
a = \sin^{2}(\frac{\Delta \phi}{2}) + \cos \phi_1 \cdot \cos \phi_2 \cdot \sin^{2}(\frac{\Delta \lambda}{2})
$$
$$
c = 2 \cdot \arctan 2(\sqrt{a}, \sqrt{1-a})
$$
$$
d = \textrm{R} \cdot c
$$
Where $\phi$ is latitude, $\lambda$ is longitude, and $\textrm{R}$ is the radius of Earth (approximately 6,371km). Additionally, $a$ is the haversine formula, which determines the haversine of the central angle connecting both points through the Earth's centre. $c$ is the haversine function, which determines the central angle. $d$ calculates the distance between both points using the radius of the Earth.

In [277]:
dist_to_offices = {} # save the distances to each office in a dictionary with the city as the key, and a dictionary of offices as the value

for city in city_coords.keys():
    temp = {} # save the distance to an office in a dictionary with the office as the key
    for office in SK_offices.keys():
        temp[office] = int(haversine((city_coords[city]['lat'], city_coords[city]['lng']), (SK_offices[office]['lat'], SK_offices[office]['lng'])))
    dist_to_offices[city] = temp

### Adding calculated data ###
Add the distance to each Sparke & Keane office from each company to the dataset.

In [278]:
add_data = {}
for i in range(len(data['Company Entity Id'])):
    if data['City'][i] != '-':
        city_country = data['City'][i] + ', ' + data['Country'][i] # combine the city and country data to facilitate searching the dist_to_offices dictionary
        temp = {}
        for office in dist_to_offices[city_country].keys():
            temp[f'afstand tot {office}'] = dist_to_offices[city_country][office]
        add_data[data['Company Entity Id'][i]] = temp

add_data_df = pd.DataFrame.from_dict(add_data, orient='index') # convert the dictionary to a DataFrame
data = data.join(add_data_df, on='Company Entity Id')

### Saving the data back to Excel ###

In [279]:
data.to_excel(output_file, sheet_name=full_data_sheet, index_label="index")

## Per office output ##
Create per-office sheets in the Excel file where all companies that meet CSRD (and Sparke & Keane) criteria are saved.

### Criteria sheet ###
Set up a sheet in the Excel file where the filter criteria (CSRD and Sparke & Keane) are specified.

In [402]:
wb = openpyxl.load_workbook(output_file)
wb.create_sheet(criteria_sheet)

i = 1
for crit in criteria.keys():
    wb[criteria_sheet][f'A{i}'] = crit
    wb[criteria_sheet][f'B{i}'] = criteria[crit]
    i += 1

### Office sheets ###
Set up the office sheets. Each sheet shows which companies meet the criteria for that specific office. For a company to be included it has to meet at least 2 out of the 3 CSRD criteria, as well as the Sparke & Keane criteria of distance to an office. The logic gate for meeting at least 2 out of the 3 CSRD criteria looks as follows: 
$$a\text{ } \&\&\text{ } (b || c)\text{ } ||\text{ } (b \&\& c)$$
This will be build into the Excel file as a Filter function, so future users can make dynamic changes if they wish.

<span style="color:green">**NOTE: This is not future proof to a changing number of CSRD criteria, or a changing logic to qualify (i.e., if 2 out of 3 criteria changes, this won't change with).**</span>

Make a dictionary for the upper case alphabet to use for easier referencing of Excel cels.

In [370]:
alph = list(string.ascii_uppercase)
alph_dict = {}
for i in range(26):
    alph_dict[i] = alph[i]

Set up the logic gates. One logic gate for each CSRD criteria, and $n$ number of logic gates for Sparke & Keane offices (as each office gets its own sheet).

In [403]:
a = f"('{full_data_sheet}'!{alph_dict[list(data.columns).index(csrd_criteria_cols[list(criteria)[0]]) + 1]}2:{alph_dict[list(data.columns).index(csrd_criteria_cols[list(criteria)[0]]) + 1]}{len(data) + 1}>='{criteria_sheet}'!B1)" # logic gate for the first csrd criteria
b = f"('{full_data_sheet}'!{alph_dict[list(data.columns).index(csrd_criteria_cols[list(criteria)[1]]) + 1]}2:{alph_dict[list(data.columns).index(csrd_criteria_cols[list(criteria)[1]]) + 1]}{len(data) + 1}>='{criteria_sheet}'!B2)" # logic gate for the second csrd criteria
c = f"('{full_data_sheet}'!{alph_dict[list(data.columns).index(csrd_criteria_cols[list(criteria)[2]]) + 1]}2:{alph_dict[list(data.columns).index(csrd_criteria_cols[list(criteria)[2]]) + 1]}{len(data) + 1}>='{criteria_sheet}'!B3)" # logic gate for the third csrd criteria

office_logic_gate = {}
for office in SK_offices:
    office_logic_gate[office] = f"(('{full_data_sheet}'!{alph_dict[list(data.columns).index(f'afstand tot {office}') + 1]}2:{alph_dict[list(data.columns).index(f'afstand tot {office}') + 1]}{len(data) + 1}<'{criteria_sheet}'!B4)*('{full_data_sheet}'!{alph_dict[list(data.columns).index(f'afstand tot {office}') + 1]}2:{alph_dict[list(data.columns).index(f'afstand tot {office}') + 1]}{len(data) + 1}>0))" # logic gate for the first S&K criteria

Fill in the office sheets with the formulas.

In [ ]:
for office in SK_offices.keys():
    wb.create_sheet(office)
    wb[office]['A1'] = f"'{full_data_sheet}'!$A$1:${alph_dict[len(data.columns)]}$1"
    wb[office]['A2'] = f"FILTER('{full_data_sheet}'!$A$2:${alph_dict[len(data.columns)]}${len(data) + 1};(((({a}*({b}+{c}))+({b}*{c})))*{office_logic_gate[office]}))"

<span style="color:green">**NOTE: because of a support issue regarding Dynamic Arrays and openpyxl the above formulas cannot yet be entered as official formulas. Instead they are entered as text. Adding "=" before them in Excel enables the formulas (with Dynamic Arrays).**</span>


### Save the ouput file ###

In [405]:
wb.save(output_file)